In [1]:
import os
from dotenv import load_dotenv
from notion_client import Client
from rich import print


load_dotenv()
notion_token = os.getenv("NOTION_TOKEN")
database_id = "54ab647b7e4949d4972d4d5ede8b48ce"

In [2]:
notion_client = Client(auth=notion_token)

# List database item

In [4]:
database_id = "54ab647b7e4949d4972d4d5ede8b48ce"
print(
    notion_client.databases.query(
        **{
            "database_id": database_id,
        }
    )
)

{
    'object': 'list',
    'results': [
        {
            'object': 'page',
            'id': '6e9474c1-f9cf-414d-87e7-73310cfafee7',
            'created_time': '2024-02-20T15:55:00.000Z',
            'last_edited_time': '2024-02-26T16:33:00.000Z',
            'created_by': {'object': 'user', 'id': 'fdc2f47b-65b2-4814-9205-33acf8bd84b6'},
            'last_edited_by': {'object': 'user', 'id': 'fdc2f47b-65b2-4814-9205-33acf8bd84b6'},
            'cover': None,
            'icon': {
                'type': 'file',
                'file': {
                    'url': 
'https://prod-files-secure.s3.us-west-2.amazonaws.com/c5bc22f0-2e59-4668-a08f-549891f2ec4b/de15b5fe-8e24-4be9-a0c2-
20744e72386f/youtube.png?X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Content-Sha256=UNSIGNED-PAYLOAD&X-Amz-Credential=AK
IAT73L2G45HZZMZUHI%2F20240227%2Fus-west-2%2Fs3%2Faws4_request&X-Amz-Date=20240227T192157Z&X-Amz-Expires=3600&X-Amz-
Signature=3cebd62b1027acbea0963c12bbdd616729216d19ae0e5ca107765e1643c2d343&X-Amz-SignedHeaders=host&x-id=GetObject'
,
                    'expiry_time': '2024-02-27T20:21:57.044Z'
                }
            },
            'parent': {'type': 'database_id', 'database_id': '54ab647b-7e49-49d4-972d-4d5ede8b48ce'},
            'archived': False,
            'properties': {
                'URL': {'id': 'Y%3C%5EN', 'type': 'url', 'url': 'https://www.youtube.com/watch?v=uzVewG8M6r0'},
                'Tags': {
                    'id': '%5Bc%3Fi',
                    'type': 'multi_select',
                    'multi_select': [
                        {'id': 'eb870446-787a-4185-81b2-bea82dd129f3', 'name': 'Yelp', 'color': 'default'},
                        {'id': 'fd1a4976-ac8c-4f04-a6e9-de93710a4290', 'name': 'ETL-Pipeline', 'color': 'gray'},
                        {'id': '40d374db-5e73-4d83-98cf-28192f037f67', 'name': 'Testing', 'color': 'pink'}
                    ]
                },
                'ID': {'id': 'wo%3E%7B', 'type': 'unique_id', 'unique_id': {'prefix': 'DD', 'number': 3}},
                'Archived': {'id': 'y%5C%3AH', 'type': 'checkbox', 'checkbox': False},
                'Source_Type': {
                    'id': '%7DnFP',
                    'type': 'select',
                    'select': {'id': '49a7b7ea-9f16-4ce8-8202-87191fcdd95f', 'name': 'YouTube', 'color': 'red'}
                },
                'Name': {
                    'id': 'title',
                    'type': 'title',
                    'title': [
                        {
                            'type': 'text',
                            'text': {'content': 'Learn to Efficiently Test ETL Pipelines', 'link': None},
                            'annotations': {
                                'bold': False,
                                'italic': False,
                                'strikethrough': False,
                                'underline': False,
                                'code': False,
                                'color': 'default'
                            },
                            'plain_text': 'Learn to Efficiently Test ETL Pipelines',
                            'href': None
                        }
                    ]
                }
            },
            'url': 
'https://www.notion.so/Learn-to-Efficiently-Test-ETL-Pipelines-6e9474c1f9cf414d87e773310cfafee7',
            'public_url': None
        }
    ],
    'next_cursor': None,
    'has_more': False,
    'type': 'page_or_database',
    'page_or_database': {},
    'request_id': '06565b70-f038-402a-afc6-399cd50c3176'
}

# Create pages

In [9]:
from notion_toolkit.schema.rich_text import RichText
from rich import print

print(RichText(type="text", plain_text="test123"))

RichText(
    type=<RichTextObjectType.TEXT: 'text'>,
    text=None,
    mention=None,
    equation=None,
    annotations=Annotations(
        bold=False,
        italic=False,
        strikethrough=False,
        underline=False,
        code=False,
        color=<Color.DEFAULT: 'default'>
    ),
    plain_text='test123',
    href=None
)

In [17]:
import os
from enum import Enum
from pydantic import BaseModel, Field, HttpUrl
from notion_toolkit.schema.rich_text import RichText
from notion_toolkit.schema.parent import DatabaseParent
from rich import print
from notion_client import Client
from dotenv import load_dotenv

class SourceType(str, Enum):
    YOUTUBE = "Youtube"
    GITHUB = "Github"
    WEBPAGE = "Webpage"


class Title(BaseModel):
    id: str = "title"
    type: str = "title"
    title: RichText


class Checkbox(BaseModel):
    checkbox: bool


class Tag(BaseModel):
    name: str


class MultiSelect(BaseModel):
    multi_select: list[Tag]


class Select(BaseModel):
    select: Tag


class SourceTypeSelect(Select):
    select: str | SourceType


class URL(BaseModel):
    url: HttpUrl


class Icon(BaseModel):
    type: str = "emoji"
    emoji: str | None = Field(
        default=None, description="The emoji character.", examples=["😻"]
    )


class SourceDBPageProperties(BaseModel):
    """Source database page properties."""

    Name: Title
    Archived: Checkbox
    Tags: MultiSelect
    Source_Type: SourceTypeSelect
    Source_URL: URL

    @classmethod
    def create_properties(
        cls,
        content: str = "",
        title_link: str | None = None,
        checkbox: bool = False,
        tags: list[str] | None = None,
        source_type: str | SourceType | None = None,
        url: str | None = None,
    ) -> "SourceDBPageProperties":
        return SourceDBPageProperties(
            Name=Title(
                title=RichText.create_for_text(content=content, link=title_link)
            ),
            Archived=Checkbox(checkbox=checkbox),
            Tags=MultiSelect(multi_select=[Tag(name=tag) for tag in tags]),
            Source_Type=SourceTypeSelect(select=source_type),
            Source_URL=URL(url=url),
        )

class SourceDBPageRequestBody(BaseModel):
    """Source database page request body."""
    
    parent: DatabaseParent
    properties: SourceDBPageProperties
    icon: Icon

    @classmethod
    def create_request_body(
        cls,
        database_id: str,
        content: str = "",
        title_link: str | None = None,
        checkbox: bool = False,
        tags: list[str] | None = None,
        source_type: str | SourceType | None = None,
        url: str | None = None,
        icon_emoji: str | None = None,
    ) -> "SourceDBPageRequestBody":
        return SourceDBPageRequestBody(
            parent=DatabaseParent(database_id=database_id),
            properties=SourceDBPageProperties.create_properties(
                content=content,
                title_link=title_link,
                checkbox=checkbox,
                tags=tags,
                source_type=source_type,
                url=url,
            ),
            icon=Icon(type="emoji", emoji=icon_emoji),
        )

load_dotenv()
notion_token = os.getenv("NOTION_TOKEN")
notion_client = Client(auth=notion_token)
database_id = "54ab647b7e4949d4972d4d5ede8b48ce"
youtube_url = "https://www.youtube.com/watch?v=JZFTDCE1plc"
video_id = "JZFTDCE1plc"
video_title = "Alexander Hendorf - ✨ FastAPI facts we wish we'd known beforehand. | PyData London 2023"
json_str = SourceDBPageRequestBody.create_request_body(
    database_id=database_id,
    content=video_title,
    checkbox=False,
    tags=["test_tag1", "test_tag2"],
    source_type="Youtube",
    url=youtube_url,
).model_dump_json(exclude_none=True)

import json

request_body = json.loads(json_str)
print(request_body)
print()
page = notion_client.pages.create(request_body)
print(page)


{
    'parent': {'type': 'database_id', 'database_id': '54ab647b-7e49-49d4-972d-4d5ede8b48ce'},
    'properties': {
        'Name': {
            'id': 'title',
            'type': 'title',
            'title': {
                'type': 'text',
                'text': {
                    'content': "Alexander Hendorf - ✨ FastAPI facts we wish we'd known beforehand. | PyData London
2023",
                    'link': {}
                },
                'annotations': {
                    'bold': False,
                    'italic': False,
                    'strikethrough': False,
                    'underline': False,
                    'code': False,
                    'color': 'default'
                },
                'plain_text': "Alexander Hendorf - ✨ FastAPI facts we wish we'd known beforehand. | PyData London 
2023"
            }
        },
        'Archived': {'checkbox': False},
        'Tags': {'multi_select': [{'name': 'test_tag1'}, {'name': 'test_tag2'}]},
        'Source_Type': {'select': 'Youtube'},
        'Source_URL': {'url': 'https://www.youtube.com/watch?v=JZFTDCE1plc'}
    },
    'icon': {'type': 'emoji'}
}

TypeError: PagesEndpoint.create() takes 1 positional argument but 2 were given

In [28]:
youtube_url = "https://www.youtube.com/watch?v=JZFTDCE1plc"
video_id = "JZFTDCE1plc"
video_title = "Alexander Hendorf - ✨ FastAPI facts we wish we'd known beforehand. | PyData London 2023"

# create a new page with a new video properties
properties = {
    "Name": {
        "id": "title",
        "type": "title",
        "title": [
            {
                "type": "text",
                "text": {"content": video_title, "link": None},
                "annotations": {
                    "bold": False,
                    "italic": False,
                    "strikethrough": False,
                    "underline": False,
                    "code": False,
                    "color": "default",
                },
                "plain_text": video_title,
                "href": None,
            }
        ],
    },
    "Archived": {"checkbox": False},
    "Tags": {"type": "multi_select", "multi_select": [{"name": "Testing"}]},
    "Source_Type": {"type": "select", "select": {"name": "YouTube"}},
    "URL": {"url": youtube_url},
}
# icon = {"icon": {"type": "emoji", "emoji": "🥑"}}
icon = {"type": "emoji", "emoji": "🎥"}
page = notion_client.pages.create(
    parent={"database_id": database_id}, properties=properties, icon=icon
)
print(page)

# Append block to the created page
page_id = page["id"]
print(f"{page_id = }")

{
    'object': 'page',
    'id': 'ffe6581e-121d-4df6-aa99-ade49ba2cbb5',
    'created_time': '2024-03-02T20:16:00.000Z',
    'last_edited_time': '2024-03-02T20:16:00.000Z',
    'created_by': {'object': 'user', 'id': 'bf31a2a3-cdd4-4e38-9816-0317dd32729c'},
    'last_edited_by': {'object': 'user', 'id': 'bf31a2a3-cdd4-4e38-9816-0317dd32729c'},
    'cover': None,
    'icon': {'type': 'emoji', 'emoji': '🎥'},
    'parent': {'type': 'database_id', 'database_id': '54ab647b-7e49-49d4-972d-4d5ede8b48ce'},
    'archived': False,
    'properties': {
        'URL': {'id': 'Y%3C%5EN', 'type': 'url', 'url': 'https://www.youtube.com/watch?v=JZFTDCE1plc'},
        'Tags': {
            'id': '%5Bc%3Fi',
            'type': 'multi_select',
            'multi_select': [{'id': '40d374db-5e73-4d83-98cf-28192f037f67', 'name': 'Testing', 'color': 'pink'}]
        },
        'ID': {'id': 'wo%3E%7B', 'type': 'unique_id', 'unique_id': {'prefix': 'DD', 'number': 17}},
        'Archived': {'id': 'y%5C%3AH', 'type': 'checkbox', 'checkbox': False},
        'Source_Type': {
            'id': '%7DnFP',
            'type': 'select',
            'select': {'id': '49a7b7ea-9f16-4ce8-8202-87191fcdd95f', 'name': 'YouTube', 'color': 'red'}
        },
        'Name': {
            'id': 'title',
            'type': 'title',
            'title': [
                {
                    'type': 'text',
                    'text': {
                        'content': "Alexander Hendorf - ✨ FastAPI facts we wish we'd known beforehand. | PyData 
London 2023",
                        'link': None
                    },
                    'annotations': {
                        'bold': False,
                        'italic': False,
                        'strikethrough': False,
                        'underline': False,
                        'code': False,
                        'color': 'default'
                    },
                    'plain_text': "Alexander Hendorf - ✨ FastAPI facts we wish we'd known beforehand. | PyData 
London 2023",
                    'href': None
                }
            ]
        }
    },
    'url': 
'https://www.notion.so/Alexander-Hendorf-FastAPI-facts-we-wish-we-d-known-beforehand-PyData-London-2023-ffe6581e121
d4df6aa99ade49ba2cbb5',
    'public_url': None,
    'request_id': '3e5ec0f1-5129-4c61-91cd-d6df5faeb683'
}

page_id = 'ffe6581e-121d-4df6-aa99-ade49ba2cbb5'

# Append content to page

In [25]:
page_content = """
# Documenting Python Projects with Sphinx and Read the Docs

## Agenda
在這個由Christopher指導的課程中，您將學習到關於Sphinx文件生成工具、RST標記語言的使用、在您的程式碼中使用Pydoc註解，
以及最後如何在Read the Docs托管網站上發布您的文件。

## Introduction to Sphinx
- **Overview**: Sphinx是Python的事實上的文件工具，利用docutils庫和基於文本的標記格式。
- **Features**:
  - 可插拔介面與擴展，例如用於解析Pydoc註解的auto-doc。
  - 支援生成多種格式的文件，如HTML、PDF和man頁面。
  - 自動創建目錄和索引。
  - 主題支援，以修改文件的外觀。
  - 默認使用reStructuredText (RST)作為標記語言，並提供了markdown (Myst)的擴展。

## Getting Started with Sphinx
- **Installation**: Sphinx可以通過PyPI獲得，並可以在虛擬環境中使用pip安裝。
- **Sphinx Quickstart**:
  - 運行`sphinx-quickstart`命令會啟動一系列配置問題，用於設置文件結構。
  - 根據用戶偏好，允許分離或合併源代碼和構建目錄。
  - 配置允許指定項目名稱、作者、版本號和文件語言。
- **Documentation Directory Structure**:
  - 生成的結構包括一個make文件、構建和源目錄、一個配置腳本(`conf.py`)，以及一個以RST格式的索引頁面。
- **Configuration and Index File**:
  - `conf.py`包含在快速啟動過程中設置的變量，並且可以進一步自定義。
  - 索引文件(`index.rst`)作為文件的首頁，允許添加節標題、註釋和指令來組織內容。

## Building Your First Document with Sphinx
- **Building Process**:
  - 使用make文件（Unix）或批處理文件（Windows）以所需格式生成文件，其中HTML是最常用的格式。
  - 構建過程創建了指定格式的結構化輸出，包括用於搜索功能等功能的額外文件。
- **Viewing Generated Documentation**:
  - 初始文件可能很簡單，但提供了添加詳細內容並在網絡瀏覽器中查看結構化輸出的基礎。

## Next Steps
- 關於利用RST進行文件結構和向您的文件項目添加內容的詳細操作指南。
- 展示如何將Pydoc註解納入您的文件以增強代碼文件，包括單元測試文件。

## Conclusion
這個課程為希望有效使用Sphinx和Read the 
Docs記錄其項目的Python開發人員提供了全面的指南。在Christopher的指導下，您將能夠製作出提高項目可用性和理解度的專業級
文件。
"""

In [27]:
print(repr(page_content))

'\n# Documenting Python Projects with Sphinx and Read the Docs\n\n## 
Agenda\n在這個由Christopher指導的課程中，您將學習到關於Sphinx文件生成工具、RST標記語言的使用、在您的程式碼中使用Pyd
oc註解，\n以及最後如何在Read the Docs托管網站上發布您的文件。\n\n## Introduction to Sphinx\n- **Overview**: 
Sphinx是Python的事實上的文件工具，利用docutils庫和基於文本的標記格式。\n- **Features**:\n  - 
可插拔介面與擴展，例如用於解析Pydoc註解的auto-doc。\n  - 支援生成多種格式的文件，如HTML、PDF和man頁面。\n  - 
自動創建目錄和索引。\n  - 主題支援，以修改文件的外觀。\n  - 默認使用reStructuredText 
(RST)作為標記語言，並提供了markdown (Myst)的擴展。\n\n## Getting Started with Sphinx\n- **Installation**: 
Sphinx可以通過PyPI獲得，並可以在虛擬環境中使用pip安裝。\n- **Sphinx Quickstart**:\n  - 
運行`sphinx-quickstart`命令會啟動一系列配置問題，用於設置文件結構。\n  - 
根據用戶偏好，允許分離或合併源代碼和構建目錄。\n  - 配置允許指定項目名稱、作者、版本號和文件語言。\n- 
**Documentation Directory Structure**:\n  - 
生成的結構包括一個make文件、構建和源目錄、一個配置腳本(`conf.py`)，以及一個以RST格式的索引頁面。\n- **Configuration
and Index File**:\n  - `conf.py`包含在快速啟動過程中設置的變量，並且可以進一步自定義。\n  - 
索引文件(`index.rst`)作為文件的首頁，允許添加節標題、註釋和指令來組織內容。\n\n## Building Your First Document with
Sphinx\n- **Building Process**:\n  - 
使用make文件（Unix）或批處理文件（Windows）以所需格式生成文件，其中HTML是最常用的格式。\n  - 
構建過程創建了指定格式的結構化輸出，包括用於搜索功能等功能的額外文件。\n- **Viewing Generated Documentation**:\n  -
初始文件可能很簡單，但提供了添加詳細內容並在網絡瀏覽器中查看結構化輸出的基礎。\n\n## Next Steps\n- 
關於利用RST進行文件結構和向您的文件項目添加內容的詳細操作指南。\n- 
展示如何將Pydoc註解納入您的文件以增強代碼文件，包括單元測試文件。\n\n## 
Conclusion\n這個課程為希望有效使用Sphinx和Read the 
\nDocs記錄其項目的Python開發人員提供了全面的指南。在Christopher的指導下，您將能夠製作出提高項目可用性和理解度的專業
級\n文件。\n'

In [21]:
page_id = "ffe6581e-121d-4df6-aa99-ade49ba2cbb5"
notion_client.blocks.children.append(
    block_id=page_id,
    children=[
        {
            "object": "block",
            "type": "paragraph",
            "paragraph": {
                "rich_text": [
                    {
                        "type": "text",
                        "text": {"content": page_content, "link": None},
                    },
                ],
            },
        }
    ],
)

{'object': 'list',
 'results': [{'object': 'block',
   'id': 'a1ed55b4-7259-4a8d-8bc7-e241396e773f',
   'parent': {'type': 'page_id',
    'page_id': 'cf69568c-31b6-414b-b438-d4f76a4208a3'},
   'created_time': '2024-02-28T14:52:00.000Z',
   'last_edited_time': '2024-02-28T14:52:00.000Z',
   'created_by': {'object': 'user',
    'id': 'bf31a2a3-cdd4-4e38-9816-0317dd32729c'},
   'last_edited_by': {'object': 'user',
    'id': 'bf31a2a3-cdd4-4e38-9816-0317dd32729c'},
   'has_children': False,
   'archived': False,
   'type': 'paragraph',
   'paragraph': {'rich_text': [{'type': 'text',
      'text': {'content': '\n# Documenting Python Projects with Sphinx and Read the Docs\n\n## Agenda\n在這個由Christopher指導的課程中，您將學習到關於Sphinx文件生成工具、RST標記語言的使用、在您的程式碼中使用Pydoc註解，\n以及最後如何在Read the Docs托管網站上發布您的文件。\n\n## Introduction to Sphinx\n- **Overview**: Sphinx是Python的事實上的文件工具，利用docutils庫和基於文本的標記格式。\n- **Features**:\n  - 可插拔介面與擴展，例如用於解析Pydoc註解的auto-doc。\n  - 支援生成多種格式的文件，如HTML、PDF和man頁面。\n  - 自動創建目錄和索引。\n  - 主題支援

In [ ]:
def create_block(block_id, content, type, **kwargs):
    notion_client.blocks.children.append(
        block_id=block_id,
        children=[
            {
                "object": "block",
                "type": type,
                type: {
                    "rich_text": [
                        {
                            "type": "text",
                            "text": {"content": content, "link": None},
                        },
                    ],
                },
                **kwargs,
            }
        ],
    )

In [24]:
page_id = "cf69568c-31b6-414b-b438-d4f76a4208a3"
notion_client.blocks.children.append(
    block_id=page_id,
    children=[
        {
            "object": "block",
            "type": "bulleted_list_item",
            "bulleted_list_item": {
                "rich_text": [
                    {
                        "type": "text",
                        "text": {"content": "Features", "link": None},
                        "annotations": {
                            "bold": True,
                            "italic": False,
                            "strikethrough": False,
                            "underline": False,
                            "code": False,
                            "color": "green",
                        },
                    },
                ],
                "color": "default",
                "children": [
                    {
                        "type": "bulleted_list_item",
                        "bulleted_list_item": {
                            "rich_text": [
                                {
                                    "type": "text",
                                    "text": {
                                        "content": "可插拔介面與擴展，例如用於解析Pydoc註解的auto-doc。",
                                        "link": None,
                                    },
                                },
                            ],
                            "color": "default",
                        },
                    }
                ],
            },
        }
    ],
)

{'object': 'list',
 'results': [{'object': 'block',
   'id': 'bd7a17c9-b87d-4e9c-9b3e-52de0cfc64e5',
   'parent': {'type': 'page_id',
    'page_id': 'cf69568c-31b6-414b-b438-d4f76a4208a3'},
   'created_time': '2024-02-28T19:59:00.000Z',
   'last_edited_time': '2024-02-28T19:59:00.000Z',
   'created_by': {'object': 'user',
    'id': 'bf31a2a3-cdd4-4e38-9816-0317dd32729c'},
   'last_edited_by': {'object': 'user',
    'id': 'bf31a2a3-cdd4-4e38-9816-0317dd32729c'},
   'has_children': True,
   'archived': False,
   'type': 'bulleted_list_item',
   'bulleted_list_item': {'rich_text': [{'type': 'text',
      'text': {'content': 'Features', 'link': None},
      'annotations': {'bold': True,
       'italic': False,
       'strikethrough': False,
       'underline': False,
       'code': False,
       'color': 'green'},
      'plain_text': 'Features',
      'href': None}],
    'color': 'default'}}],
 'next_cursor': None,
 'has_more': False,
 'type': 'block',
 'block': {},
 'request_id': 'ddd85e4